In [ ]:
from cre import *
from parser import get_parser
from dataset import get_dataset

In [ ]:
args = get_parser().parse_args(args=[])

dataset = get_dataset(args)

model = CRE(args)
model.fit(dataset)
model.plot()

ite = model.eval(dataset)